http://enr.elections.ca/ElectoralDistricts.aspx?lang=e

In [5]:
import requests
import pandas as pd
import csvkit as csv
import re

In [6]:
VOTES = 'Votes obtained - Votes obtenus'
DISTRICT_NAME = 'Electoral district name'
PARTY = 'Political affiliation'
ridings = {
  "Banff--Airdrie" : { "id" : 1606, "province" : "Alberta" },
  "Battle River--Crowfoot" : { "id" : 1607, "province" : "Alberta" },
  "Bow River" : { "id" : 1608, "province" : "Alberta" },
  "Calgary Centre" : { "id" : 1609, "province" : "Alberta" },
  "Calgary Confederation" : { "id" : 1610, "province" : "Alberta" },
  "Calgary Forest Lawn" : { "id" : 1611, "province" : "Alberta" },
  "Calgary Heritage" : { "id" : 1612, "province" : "Alberta" },
  "Calgary Midnapore" : { "id" : 1613, "province" : "Alberta" },
  "Calgary Nose Hill" : { "id" : 1614, "province" : "Alberta" },
  "Calgary Rocky Ridge" : { "id" : 1615, "province" : "Alberta" },
  "Calgary Shepard" : { "id" : 1616, "province" : "Alberta" },
  "Calgary Signal Hill" : { "id" : 1617, "province" : "Alberta" },
  "Calgary Skyview" : { "id" : 1618, "province" : "Alberta" },
  "Edmonton Centre" : { "id" : 1619, "province" : "Alberta" },
  "Edmonton Griesbach" : { "id" : 1620, "province" : "Alberta" },
  "Edmonton Manning" : { "id" : 1621, "province" : "Alberta" },
  "Edmonton Mill Woods" : { "id" : 1622, "province" : "Alberta" },
  "Edmonton Riverbend" : { "id" : 1623, "province" : "Alberta" },
  "Edmonton Strathcona" : { "id" : 1624, "province" : "Alberta" },
  "Edmonton West" : { "id" : 1625, "province" : "Alberta" },
  "Edmonton--Wetaskiwin" : { "id" : 1626, "province" : "Alberta" },
  "Foothills" : { "id" : 1627, "province" : "Alberta" },
  "Fort McMurray--Cold Lake" : { "id" : 1628, "province" : "Alberta" },
  "Grande Prairie--Mackenzie" : { "id" : 1629, "province" : "Alberta" },
  "Lakeland" : { "id" : 1630, "province" : "Alberta" },
  "Lethbridge" : { "id" : 1631, "province" : "Alberta" },
  "Medicine Hat--Cardston--Warner" : { "id" : 1632, "province" : "Alberta" },
  "Peace River--Westlock" : { "id" : 1633, "province" : "Alberta" },
  "Red Deer--Lacombe" : { "id" : 1635, "province" : "Alberta" },
  "Red Deer--Mountain View" : { "id" : 1634, "province" : "Alberta" },
  "Sherwood Park--Fort Saskatchewan" : { "id" : 1636, "province" : "Alberta" },
  "St. Albert--Edmonton" : { "id" : 1637, "province" : "Alberta" },
  "Sturgeon River--Parkland" : { "id" : 1638, "province" : "Alberta" },
  "Yellowhead" : { "id" : 1639, "province" : "Alberta" },
  "Abbotsford" : { "id" : 1674, "province" : "Alberta" },
  "Burnaby North--Seymour" : { "id" : 1675, "province" : "British Columbia" },
  "Burnaby South" : { "id" : 1676, "province" : "British Columbia" },
  "Cariboo--Prince George" : { "id" : 1677, "province" : "British Columbia" },
  "Central Okanagan--Similkameen--Nicola" : { "id" : 1678, "province" : "British Columbia" },
  "Chilliwack--Hope" : { "id" : 1679, "province" : "British Columbia" },
  "Cloverdale--Langley City" : { "id" : 1680, "province" : "British Columbia" },
  "Coquitlam--Port Coquitlam" : { "id" : 1681, "province" : "British Columbia" },
  "Courtenay--Alberni" : { "id" : 1682, "province" : "British Columbia" },
  "Cowichan--Malahat--Langford" : { "id" : 1683, "province" : "British Columbia" },
  "Delta" : { "id" : 1684, "province" : "British Columbia" },
  "Esquimalt--Saanich--Sooke" : { "id" : 1699, "province" : "British Columbia" },
  "Fleetwood--Port Kells" : { "id" : 1685, "province" : "British Columbia" },
  "Kamloops--Thompson--Cariboo" : { "id" : 1686, "province" : "British Columbia" },
  "Kelowna--Lake Country" : { "id" : 1687, "province" : "British Columbia" },
  "Kootenay--Columbia" : { "id" : 1688, "province" : "British Columbia" },
  "Langley--Aldergrove" : { "id" : 1689, "province" : "British Columbia" },
  "Mission--Matsqui--Fraser Canyon" : { "id" : 1690, "province" : "British Columbia" },
  "Nanaimo--Ladysmith" : { "id" : 1691, "province" : "British Columbia" },
  "New Westminster--Burnaby" : { "id" : 1692, "province" : "British Columbia" },
  "North Island--Powell River" : { "id" : 1710, "province" : "British Columbia" },
  "North Okanagan--Shuswap" : { "id" : 1693, "province" : "British Columbia" },
  "North Vancouver" : { "id" : 1694, "province" : "British Columbia" },
  "Pitt Meadows--Maple Ridge" : { "id" : 1695, "province" : "British Columbia" },
  "Port Moody--Coquitlam" : { "id" : 1696, "province" : "British Columbia" },
  "Prince George--Peace River--Northern Rockies" : { "id" : 1697, "province" : "British Columbia" },
  "Richmond Centre" : { "id" : 1698, "province" : "British Columbia" },
  "Saanich--Gulf Islands" : { "id" : 1700, "province" : "British Columbia" },
  "Skeena--Bulkley Valley" : { "id" : 1701, "province" : "British Columbia" },
  "South Okanagan--West Kootenay" : { "id" : 1702, "province" : "British Columbia" },
  "South Surrey--White Rock" : { "id" : 1703, "province" : "British Columbia" },
  "Steveston--Richmond East" : { "id" : 1704, "province" : "British Columbia" },
  "Surrey Centre" : { "id" : 1705, "province" : "British Columbia" },
  "Surrey--Newton" : { "id" : 1706, "province" : "British Columbia" },
  "Vancouver Centre" : { "id" : 1707, "province" : "British Columbia" },
  "Vancouver East" : { "id" : 1708, "province" : "British Columbia" },
  "Vancouver Granville" : { "id" : 1709, "province" : "British Columbia" },
  "Vancouver Kingsway" : { "id" : 1711, "province" : "British Columbia" },
  "Vancouver Quadra" : { "id" : 1712, "province" : "British Columbia" },
  "Vancouver South" : { "id" : 1713, "province" : "British Columbia" },
  "Victoria" : { "id" : 1714, "province" : "British Columbia" },
  "West Vancouver--Sunshine Coast--Sea to Sky Country" : { "id" : 1715, "province" : "British Columbia" },
  "Brandon--Souris" : { "id" : 1592, "province" : "Manitoba" },
  "Charleswood--St. James--Assiniboia--Headingley" : { "id" : 1593, "province" : "Manitoba" },
  "Churchill--Keewatinook Aski" : { "id" : 1594, "province" : "Manitoba" },
  "Dauphin--Swan River--Neepawa" : { "id" : 1595, "province" : "Manitoba" },
  "Elmwood--Transcona" : { "id" : 1596, "province" : "Manitoba" },
  "Kildonan--St. Paul" : { "id" : 1597, "province" : "Manitoba" },
  "Portage--Lisgar" : { "id" : 1598, "province" : "Manitoba" },
  "Provencher" : { "id" : 1599, "province" : "Manitoba" },
  "Saint Boniface--Saint Vital" : { "id" : 1600, "province" : "Manitoba" },
  "Selkirk--Interlake--Eastman" : { "id" : 1601, "province" : "Manitoba" },
  "Winnipeg Centre" : { "id" : 1602, "province" : "Manitoba" },
  "Winnipeg North" : { "id" : 1603, "province" : "Manitoba" },
  "Winnipeg South" : { "id" : 1604, "province" : "Manitoba" },
  "Winnipeg South Centre" : { "id" : 1605, "province" : "Manitoba" },
  "Acadie--Bathurst" : { "id" : 1582, "province" : "New Brunswick" },
  "Beauséjour" : { "id" : 1583, "province" : "New Brunswick" },
  "Fredericton" : { "id" : 1584, "province" : "New Brunswick" },
  "Fundy Royal" : { "id" : 1585, "province" : "New Brunswick" },
  "Madawaska--Restigouche" : { "id" : 1586, "province" : "New Brunswick" },
  "Miramichi--Grand Lake" : { "id" : 1587, "province" : "New Brunswick" },
  "Moncton--Riverview--Dieppe" : { "id" : 1588, "province" : "New Brunswick" },
  "New Brunswick Southwest" : { "id" : 1589, "province" : "New Brunswick" },
  "Saint John--Rothesay" : { "id" : 1590, "province" : "New Brunswick" },
  "Tobique--Mactaquac" : { "id" : 1591, "province" : "New Brunswick" },
  "Avalon" : { "id" : 1560, "province" : "Newfoundland and Labrador" },
  "Bonavista--Burin--Trinity" : { "id" : 1561, "province" : "Newfoundland and Labrador" },
  "Coast of Bays--Central--Notre Dame" : { "id" : 1562, "province" : "Newfoundland and Labrador" },
  "Labrador" : { "id" : 1563, "province" : "Newfoundland and Labrador" },
  "Long Range Mountains" : { "id" : 1564, "province" : "Newfoundland and Labrador" },
  "St. John's East" : { "id" : 1565, "province" : "Newfoundland and Labrador" },
  "St. John's South--Mount Pearl" : { "id" : 1566, "province" : "Newfoundland and Labrador" },
  "Northwest Territories" : { "id" : 1641, "province" : "Northwest Territories" },
  "Cape Breton--Canso" : { "id" : 1571, "province" : "Nova Scotia" },
  "Central Nova" : { "id" : 1572, "province" : "Nova Scotia" },
  "Cumberland--Colchester" : { "id" : 1573, "province" : "Nova Scotia" },
  "Dartmouth--Cole Harbour" : { "id" : 1574, "province" : "Nova Scotia" },
  "Halifax" : { "id" : 1575, "province" : "Nova Scotia" },
  "Halifax West" : { "id" : 1576, "province" : "Nova Scotia" },
  "Kings--Hants" : { "id" : 1577, "province" : "Nova Scotia" },
  "Sackville--Preston--Chezzetcook" : { "id" : 1578, "province" : "Nova Scotia" },
  "South Shore--St. Margarets" : { "id" : 1579, "province" : "Nova Scotia" },
  "Sydney--Victoria" : { "id" : 1580, "province" : "Nova Scotia" },
  "West Nova" : { "id" : 1581, "province" : "Nova Scotia" },
  "Nunavut" : { "id" : 1642, "province" : "Nunavut" },
  "Ajax" : { "id" : 2148, "province" : "Ontario" },
  "Algoma--Manitoulin--Kapuskasing" : { "id" : 2149, "province" : "Ontario" },
  "Aurora--Oak Ridges--Richmond Hill" : { "id" : 2150, "province" : "Ontario" },
  "Barrie--Innisfil" : { "id" : 2151, "province" : "Ontario" },
  "Barrie--Springwater--Oro-Medonte" : { "id" : 2152, "province" : "Ontario" },
  "Bay of Quinte" : { "id" : 2153, "province" : "Ontario" },
  "Beaches--East York" : { "id" : 2154, "province" : "Ontario" },
  "Brampton Centre" : { "id" : 2155, "province" : "Ontario" },
  "Brampton East" : { "id" : 2156, "province" : "Ontario" },
  "Brampton North" : { "id" : 2157, "province" : "Ontario" },
  "Brampton South" : { "id" : 2158, "province" : "Ontario" },
  "Brampton West" : { "id" : 2159, "province" : "Ontario" },
  "Brantford--Brant" : { "id" : 2160, "province" : "Ontario" },
  "Bruce--Grey--Owen Sound" : { "id" : 2161, "province" : "Ontario" },
  "Burlington" : { "id" : 2162, "province" : "Ontario" },
  "Cambridge" : { "id" : 2163, "province" : "Ontario" },
  "Carleton" : { "id" : 2235, "province" : "Ontario" },
  "Chatham-Kent--Leamington" : { "id" : 2164, "province" : "Ontario" },
  "Davenport" : { "id" : 2165, "province" : "Ontario" },
  "Don Valley East" : { "id" : 2166, "province" : "Ontario" },
  "Don Valley North" : { "id" : 2167, "province" : "Ontario" },
  "Don Valley West" : { "id" : 2168, "province" : "Ontario" },
  "Dufferin--Caledon" : { "id" : 2169, "province" : "Ontario" },
  "Durham" : { "id" : 2170, "province" : "Ontario" },
  "Eglinton--Lawrence" : { "id" : 2171, "province" : "Ontario" },
  "Elgin--Middlesex--London" : { "id" : 2172, "province" : "Ontario" },
  "Essex" : { "id" : 2173, "province" : "Ontario" },
  "Etobicoke Centre" : { "id" : 2174, "province" : "Ontario" },
  "Etobicoke North" : { "id" : 2176, "province" : "Ontario" },
  "Etobicoke--Lakeshore" : { "id" : 2175, "province" : "Ontario" },
  "Flamborough--Glanbrook" : { "id" : 2177, "province" : "Ontario" },
  "Glengarry--Prescott--Russell" : { "id" : 2178, "province" : "Ontario" },
  "Guelph" : { "id" : 2179, "province" : "Ontario" },
  "Haldimand--Norfolk" : { "id" : 2180, "province" : "Ontario" },
  "Haliburton--Kawartha Lakes--Brock" : { "id" : 2181, "province" : "Ontario" },
  "Hamilton Centre" : { "id" : 2182, "province" : "Ontario" },
  "Hamilton East--Stoney Creek" : { "id" : 2183, "province" : "Ontario" },
  "Hamilton Mountain" : { "id" : 2184, "province" : "Ontario" },
  "Hamilton West--Ancaster--Dundas" : { "id" : 2185, "province" : "Ontario" },
  "Hastings--Lennox and Addington" : { "id" : 2186, "province" : "Ontario" },
  "Humber River--Black Creek" : { "id" : 2268, "province" : "Ontario" },
  "Huron--Bruce" : { "id" : 2187, "province" : "Ontario" },
  "Kanata--Carleton" : { "id" : 2188, "province" : "Ontario" },
  "Kenora" : { "id" : 2189, "province" : "Ontario" },
  "Kingston and the Islands" : { "id" : 2191, "province" : "Ontario" },
  "King--Vaughan" : { "id" : 2190, "province" : "Ontario" },
  "Kitchener Centre" : { "id" : 2192, "province" : "Ontario" },
  "Kitchener South--Hespeler" : { "id" : 2194, "province" : "Ontario" },
  "Kitchener--Conestoga" : { "id" : 2193, "province" : "Ontario" },
  "Lambton--Kent--Middlesex" : { "id" : 2195, "province" : "Ontario" },
  "Lanark--Frontenac--Kingston" : { "id" : 2196, "province" : "Ontario" },
  "Leeds-Grenville-Thousand Islands and Rideau Lakes" : { "id" : 2197, "province" : "Ontario" },
  "London North Centre" : { "id" : 2199, "province" : "Ontario" },
  "London West" : { "id" : 2200, "province" : "Ontario" },
  "London--Fanshawe" : { "id" : 2198, "province" : "Ontario" },
  "Markham--Stouffville" : { "id" : 2201, "province" : "Ontario" },
  "Markham--Thornhill" : { "id" : 2202, "province" : "Ontario" },
  "Markham--Unionville" : { "id" : 2203, "province" : "Ontario" },
  "Milton" : { "id" : 2204, "province" : "Ontario" },
  "Mississauga Centre" : { "id" : 2205, "province" : "Ontario" },
  "Mississauga East--Cooksville" : { "id" : 2206, "province" : "Ontario" },
  "Mississauga--Erin Mills" : { "id" : 2207, "province" : "Ontario" },
  "Mississauga--Lakeshore" : { "id" : 2208, "province" : "Ontario" },
  "Mississauga--Malton" : { "id" : 2209, "province" : "Ontario" },
  "Mississauga--Streetsville" : { "id" : 2210, "province" : "Ontario" },
  "Nepean" : { "id" : 2211, "province" : "Ontario" },
  "Newmarket--Aurora" : { "id" : 2212, "province" : "Ontario" },
  "Niagara Centre" : { "id" : 2213, "province" : "Ontario" },
  "Niagara Falls" : { "id" : 2214, "province" : "Ontario" },
  "Niagara West" : { "id" : 2215, "province" : "Ontario" },
  "Nickel Belt" : { "id" : 2216, "province" : "Ontario" },
  "Nipissing--Timiskaming" : { "id" : 2217, "province" : "Ontario" },
  "Northumberland--Peterborough South" : { "id" : 2218, "province" : "Ontario" },
  "Oakville" : { "id" : 2219, "province" : "Ontario" },
  "Oakville North--Burlington" : { "id" : 2220, "province" : "Ontario" },
  "Orléans" : { "id" : 2223, "province" : "Ontario" },
  "Oshawa" : { "id" : 2221, "province" : "Ontario" },
  "Ottawa Centre" : { "id" : 2222, "province" : "Ontario" },
  "Ottawa South" : { "id" : 2224, "province" : "Ontario" },
  "Ottawa West--Nepean" : { "id" : 2226, "province" : "Ontario" },
  "Ottawa--Vanier" : { "id" : 2225, "province" : "Ontario" },
  "Oxford" : { "id" : 2227, "province" : "Ontario" },
  "Parkdale--High Park" : { "id" : 2228, "province" : "Ontario" },
  "Parry Sound--Muskoka" : { "id" : 2229, "province" : "Ontario" },
  "Perth--Wellington" : { "id" : 2230, "province" : "Ontario" },
  "Peterborough--Kawartha" : { "id" : 2231, "province" : "Ontario" },
  "Pickering--Uxbridge" : { "id" : 2232, "province" : "Ontario" },
  "Renfrew--Nipissing--Pembroke" : { "id" : 2233, "province" : "Ontario" },
  "Richmond Hill" : { "id" : 2234, "province" : "Ontario" },
  "Sarnia--Lambton" : { "id" : 2238, "province" : "Ontario" },
  "Sault Ste. Marie" : { "id" : 2239, "province" : "Ontario" },
  "Scarborough Centre" : { "id" : 2241, "province" : "Ontario" },
  "Scarborough North" : { "id" : 2243, "province" : "Ontario" },
  "Scarborough Southwest" : { "id" : 2245, "province" : "Ontario" },
  "Scarborough--Agincourt" : { "id" : 2240, "province" : "Ontario" },
  "Scarborough--Guildwood" : { "id" : 2242, "province" : "Ontario" },
  "Scarborough--Rouge Park" : { "id" : 2244, "province" : "Ontario" },
  "Simcoe North" : { "id" : 2247, "province" : "Ontario" },
  "Simcoe--Grey" : { "id" : 2246, "province" : "Ontario" },
  "Spadina--Fort York" : { "id" : 2248, "province" : "Ontario" },
  "St. Catharines" : { "id" : 2236, "province" : "Ontario" },
  "Stormont--Dundas--South Glengarry" : { "id" : 2249, "province" : "Ontario" },
  "Sudbury" : { "id" : 2250, "province" : "Ontario" },
  "Thornhill" : { "id" : 2251, "province" : "Ontario" },
  "Thunder Bay--Rainy River" : { "id" : 2252, "province" : "Ontario" },
  "Thunder Bay--Superior North" : { "id" : 2253, "province" : "Ontario" },
  "Timmins--James Bay" : { "id" : 2254, "province" : "Ontario" },
  "Toronto Centre" : { "id" : 2255, "province" : "Ontario" },
  "Toronto--Danforth" : { "id" : 2256, "province" : "Ontario" },
  "Toronto--St. Paul's" : { "id" : 2237, "province" : "Ontario" },
  "University--Rosedale" : { "id" : 2257, "province" : "Ontario" },
  "Vaughan--Woodbridge" : { "id" : 2258, "province" : "Ontario" },
  "Waterloo" : { "id" : 2259, "province" : "Ontario" },
  "Wellington--Halton Hills" : { "id" : 2260, "province" : "Ontario" },
  "Whitby" : { "id" : 2261, "province" : "Ontario" },
  "Willowdale" : { "id" : 2262, "province" : "Ontario" },
  "Windsor West" : { "id" : 2264, "province" : "Ontario" },
  "Windsor--Tecumseh" : { "id" : 2263, "province" : "Ontario" },
  "York Centre" : { "id" : 2265, "province" : "Ontario" },
  "York South--Weston" : { "id" : 2267, "province" : "Ontario" },
  "York--Simcoe" : { "id" : 2266, "province" : "Ontario" },
  "Cardigan" : { "id" : 1567, "province" : "Prince Edward Island" },
  "Charlottetown" : { "id" : 1568, "province" : "Prince Edward Island" },
  "Egmont" : { "id" : 1569, "province" : "Prince Edward Island" },
  "Malpeque" : { "id" : 1570, "province" : "Prince Edward Island" },
  "Abitibi--Baie-James--Nunavik--Eeyou" : { "id" : 2070, "province" : "Quebec" },
  "Abitibi--Témiscamingue" : { "id" : 2071, "province" : "Quebec" },
  "Ahuntsic-Cartierville" : { "id" : 2072, "province" : "Quebec" },
  "Alfred-Pellan" : { "id" : 2073, "province" : "Quebec" },
  "Argenteuil--La Petite-Nation" : { "id" : 2074, "province" : "Quebec" },
  "Avignon--La Mitis--Matane--Matapédia" : { "id" : 2075, "province" : "Quebec" },
  "Beauce" : { "id" : 2076, "province" : "Quebec" },
  "Beauport-Côte-de-Beaupré-Île d'Orléans-Charlevoix" : { "id" : 2089, "province" : "Quebec" },
  "Beauport--Limoilou" : { "id" : 2077, "province" : "Quebec" },
  "Bécancour--Nicolet--Saurel" : { "id" : 2078, "province" : "Quebec" },
  "Bellechasse--Les Etchemins--Lévis" : { "id" : 2079, "province" : "Quebec" },
  "Beloeil--Chambly" : { "id" : 2080, "province" : "Quebec" },
  "Berthier--Maskinongé" : { "id" : 2081, "province" : "Quebec" },
  "Bourassa" : { "id" : 2084, "province" : "Quebec" },
  "Brome--Missisquoi" : { "id" : 2085, "province" : "Quebec" },
  "Brossard--Saint-Lambert" : { "id" : 2086, "province" : "Quebec" },
  "Charlesbourg--Haute-Saint-Charles" : { "id" : 2088, "province" : "Quebec" },
  "Châteauguay--Lacolle" : { "id" : 2090, "province" : "Quebec" },
  "Chicoutimi--Le Fjord" : { "id" : 2091, "province" : "Quebec" },
  "Compton--Stanstead" : { "id" : 2092, "province" : "Quebec" },
  "Dorval--Lachine--LaSalle" : { "id" : 2093, "province" : "Quebec" },
  "Drummond" : { "id" : 2094, "province" : "Quebec" },
  "Gaspésie--Les Îles-de-la-Madeleine" : { "id" : 2095, "province" : "Quebec" },
  "Gatineau" : { "id" : 2096, "province" : "Quebec" },
  "Hochelaga" : { "id" : 2097, "province" : "Quebec" },
  "Honoré-Mercier" : { "id" : 2098, "province" : "Quebec" },
  "Hull--Aylmer" : { "id" : 2099, "province" : "Quebec" },
  "Joliette" : { "id" : 2100, "province" : "Quebec" },
  "Jonquière" : { "id" : 2101, "province" : "Quebec" },
  "La Pointe-de-l'Île" : { "id" : 2102, "province" : "Quebec" },
  "La Prairie" : { "id" : 2103, "province" : "Quebec" },
  "Lac-Saint-Jean" : { "id" : 2104, "province" : "Quebec" },
  "Lac-Saint-Louis" : { "id" : 2105, "province" : "Quebec" },
  "LaSalle--Émard--Verdun" : { "id" : 2106, "province" : "Quebec" },
  "Laurentides--Labelle" : { "id" : 2107, "province" : "Quebec" },
  "Laurier--Sainte-Marie" : { "id" : 2108, "province" : "Quebec" },
  "Laval--Les Îles" : { "id" : 2109, "province" : "Quebec" },
  "Lévis--Lotbinière" : { "id" : 2111, "province" : "Quebec" },
  "Longueuil--Charles-LeMoyne" : { "id" : 2110, "province" : "Quebec" },
  "Longueuil--Saint-Hubert" : { "id" : 2112, "province" : "Quebec" },
  "Louis-Hébert" : { "id" : 2113, "province" : "Quebec" },
  "Louis-Saint-Laurent" : { "id" : 2114, "province" : "Quebec" },
  "Manicouagan" : { "id" : 2115, "province" : "Quebec" },
  "Marc-Aurèle-Fortin" : { "id" : 2134, "province" : "Quebec" },
  "Mégantic--L'Érable" : { "id" : 2116, "province" : "Quebec" },
  "Mirabel" : { "id" : 2117, "province" : "Quebec" },
  "Montarville" : { "id" : 2118, "province" : "Quebec" },
  "Montcalm" : { "id" : 2119, "province" : "Quebec" },
  "Montmagny--L'Islet--Kamouraska--Rivière-du-Loup" : { "id" : 2120, "province" : "Quebec" },
  "Mount Royal" : { "id" : 2121, "province" : "Quebec" },
  "Notre-Dame-de-Grâce--Westmount" : { "id" : 2122, "province" : "Quebec" },
  "Outremont" : { "id" : 2123, "province" : "Quebec" },
  "Papineau" : { "id" : 2124, "province" : "Quebec" },
  "Pierre-Boucher--Les Patriotes--Verchères" : { "id" : 2083, "province" : "Quebec" },
  "Pierrefonds--Dollard" : { "id" : 2125, "province" : "Quebec" },
  "Pontiac" : { "id" : 2126, "province" : "Quebec" },
  "Portneuf--Jacques-Cartier" : { "id" : 2127, "province" : "Quebec" },
  "Québec" : { "id" : 2128, "province" : "Quebec" },
  "Repentigny" : { "id" : 2129, "province" : "Quebec" },
  "Richmond--Arthabaska" : { "id" : 2130, "province" : "Quebec" },
  "Rimouski-Neigette--Témiscouata--Les Basques" : { "id" : 2087, "province" : "Quebec" },
  "Rivière-des-Mille-Îles" : { "id" : 2131, "province" : "Quebec" },
  "Rivière-du-Nord" : { "id" : 2132, "province" : "Quebec" },
  "Rosemont--La Petite-Patrie" : { "id" : 2133, "province" : "Quebec" },
  "Saint-Hyacinthe--Bagot" : { "id" : 2135, "province" : "Quebec" },
  "Saint-Jean" : { "id" : 2136, "province" : "Quebec" },
  "Saint-Laurent" : { "id" : 2137, "province" : "Quebec" },
  "Saint-Léonard--Saint-Michel" : { "id" : 2138, "province" : "Quebec" },
  "Saint-Maurice--Champlain" : { "id" : 2139, "province" : "Quebec" },
  "Salaberry--Suroît" : { "id" : 2140, "province" : "Quebec" },
  "Shefford" : { "id" : 2141, "province" : "Quebec" },
  "Sherbrooke" : { "id" : 2142, "province" : "Quebec" },
  "Terrebonne" : { "id" : 2144, "province" : "Quebec" },
  "Thérèse-De Blainville" : { "id" : 2082, "province" : "Quebec" },
  "Trois-Rivières" : { "id" : 2145, "province" : "Quebec" },
  "Vaudreuil--Soulanges" : { "id" : 2143, "province" : "Quebec" },
  "Ville-Marie--Le Sud-Ouest--Île-des-Soeurs" : { "id" : 2146, "province" : "Quebec" },
  "Vimy" : { "id" : 2147, "province" : "Quebec" },
  "Battlefords--Lloydminster" : { "id" : 1660, "province" : "Saskatchewan" },
  "Carlton Trail--Eagle Creek" : { "id" : 1663, "province" : "Saskatchewan" },
  "Cypress Hills--Grasslands" : { "id" : 1661, "province" : "Saskatchewan" },
  "Desnethé--Missinippi--Churchill River" : { "id" : 1662, "province" : "Saskatchewan" },
  "Moose Jaw--Lake Centre--Lanigan" : { "id" : 1664, "province" : "Saskatchewan" },
  "Prince Albert" : { "id" : 1665, "province" : "Saskatchewan" },
  "Regina--Lewvan" : { "id" : 1666, "province" : "Saskatchewan" },
  "Regina--Qu'Appelle" : { "id" : 1667, "province" : "Saskatchewan" },
  "Regina--Wascana" : { "id" : 1668, "province" : "Saskatchewan" },
  "Saskatoon West" : { "id" : 1671, "province" : "Saskatchewan" },
  "Saskatoon--Grasswood" : { "id" : 1669, "province" : "Saskatchewan" },
  "Saskatoon--University" : { "id" : 1670, "province" : "Saskatchewan" },
  "Souris--Moose Mountain" : { "id" : 1672, "province" : "Saskatchewan" },
  "Yorkton--Melville" : { "id" : 1673, "province" : "Saskatchewan" },
  "Yukon" : { "id" : 1640, "province" : "Yukon" },
}

In [9]:
r = requests.get('http://enr.elections.ca/DownloadResults.aspx?ed=%d&result=2' % ridings['Yukon']['id'])

In [10]:
text = r.text

In [11]:
district_num = re.compile(r'^\d{5}')
def fetchAndParseTSV(riding_name,data):
    print(riding_name)
    r = requests.get('http://enr.elections.ca/DownloadResults.aspx?ed=%d&result=2' % data['id'])
    text = r.text
    stripped = [line for line in text.splitlines()[1:] if line[:9] == 'Electoral' or re.match(district_num, line)]
    reader = csv.DictReader(stripped, delimiter='\t')
    rows = [row for row in reader]
    for row in rows:
        row['Province'] = data['province']
    return rows

In [12]:
riding_data = [fetchAndParseTSV(riding_name,riding_id) for riding_name,riding_id in ridings.items()]
riding_data[:3]

Repentigny
Prince George--Peace River--Northern Rockies
Parkdale--High Park
New Brunswick Southwest
Brandon--Souris
Grande Prairie--Mackenzie
Windsor West
Cowichan--Malahat--Langford
Thunder Bay--Superior North
Renfrew--Nipissing--Pembroke
Hamilton Mountain
North Island--Powell River
Souris--Moose Mountain
Leeds-Grenville-Thousand Islands and Rideau Lakes
Vaughan--Woodbridge
Chatham-Kent--Leamington
Elgin--Middlesex--London
Coquitlam--Port Coquitlam
Laval--Les Îles
Vaudreuil--Soulanges
Markham--Unionville
Central Nova
Hochelaga
Ville-Marie--Le Sud-Ouest--Île-des-Soeurs
Medicine Hat--Cardston--Warner
Oxford
Montarville
Don Valley East
Saskatoon--Grasswood
Brampton Centre
Durham
Sherbrooke
Pontiac
Saint-Hyacinthe--Bagot
Vancouver Kingsway
Calgary Midnapore
Beloeil--Chambly
Sarnia--Lambton
Louis-Hébert
Winnipeg North
Richmond--Arthabaska
Labrador
Kitchener South--Hespeler
Compton--Stanstead
Calgary Signal Hill
Manicouagan
Edmonton Centre
Etobicoke--Lakeshore
Calgary Rocky Ridge
Wellington

[[{'% Votes obtained - Votes obtenus %': '27.3',
   'Appartenance politique': 'Libéral',
   'Electoral district name': 'Repentigny',
   'Electoral district number - Numéro de la circonscription': '24060',
   'Given name - Prénom': 'Adriana',
   'Middle name(s) - Autre(s) prénom(s)': '',
   'Nom de la circonscription': 'Repentigny',
   'Political affiliation': 'Liberal',
   'Province': 'Quebec',
   'Rejected ballots - Bulletins rejetés***': '1179',
   'Surname - Nom de famille': 'Dudas',
   'Total number of ballots cast - Nombre total de votes déposés': '66390',
   'Type de résultats**': 'validés',
   'Type of results*': 'validated',
   'Votes obtained - Votes obtenus': '17798'},
  {'% Votes obtained - Votes obtenus %': '2',
   'Appartenance politique': 'Forces et Démocratie - Allier les forces de nos régions',
   'Electoral district name': 'Repentigny',
   'Electoral district number - Numéro de la circonscription': '24060',
   'Given name - Prénom': 'Johnathan',
   'Middle name(s) - Au

In [13]:
flat_riding_data = [v for sublist in riding_data for v in sublist]
flat_riding_data[:10]

[{'% Votes obtained - Votes obtenus %': '27.3',
  'Appartenance politique': 'Libéral',
  'Electoral district name': 'Repentigny',
  'Electoral district number - Numéro de la circonscription': '24060',
  'Given name - Prénom': 'Adriana',
  'Middle name(s) - Autre(s) prénom(s)': '',
  'Nom de la circonscription': 'Repentigny',
  'Political affiliation': 'Liberal',
  'Province': 'Quebec',
  'Rejected ballots - Bulletins rejetés***': '1179',
  'Surname - Nom de famille': 'Dudas',
  'Total number of ballots cast - Nombre total de votes déposés': '66390',
  'Type de résultats**': 'validés',
  'Type of results*': 'validated',
  'Votes obtained - Votes obtenus': '17798'},
 {'% Votes obtained - Votes obtenus %': '2',
  'Appartenance politique': 'Forces et Démocratie - Allier les forces de nos régions',
  'Electoral district name': 'Repentigny',
  'Electoral district number - Numéro de la circonscription': '24060',
  'Given name - Prénom': 'Johnathan',
  'Middle name(s) - Autre(s) prénom(s)': ''

In [14]:
riding_df = pd.DataFrame(flat_riding_data)
riding_df[['Votes obtained - Votes obtenus']] = riding_df[['Votes obtained - Votes obtenus']].astype(int)
riding_df.head(5)

,% Votes obtained - Votes obtenus %,Appartenance politique,Electoral district name,Electoral district number - Numéro de la circonscription,Given name - Prénom,Middle name(s) - Autre(s) prénom(s),Nom de la circonscription,Political affiliation,Province,Rejected ballots - Bulletins rejetés***,Surname - Nom de famille,Total number of ballots cast - Nombre total de votes déposés,Type de résultats**,Type of results*,Votes obtained - Votes obtenus
0,27.3,Libéral,Repentigny,24060,Adriana,,Repentigny,Liberal,Quebec,1179,Dudas,66390,validés,validated,17798
1,2,Forces et Démocratie - Allier les forces de no...,Repentigny,24060,Johnathan,,Repentigny,Forces et Démocratie - Allier les forces de no...,Quebec,1179,Cloutier,66390,validés,validated,1333
2,23.3,NPD-Nouveau Parti démocratique,Repentigny,24060,Réjean,,Repentigny,NDP-New Democratic Party,Quebec,1179,Bellemare,66390,validés,validated,15167
3,1.9,Parti Vert,Repentigny,24060,Yoland,,Repentigny,Green Party,Quebec,1179,Gilbert,66390,validés,validated,1242
4,34.7,Bloc Québécois,Repentigny,24060,Monique,,Repentigny,Bloc Québécois,Quebec,1179,Pauzé,66390,validés,validated,22618


In [15]:
riding_df.to_csv('./results/canada-2015.csv')

In [56]:
riding_df.groupby(['Electoral district name'])['Votes obtained - Votes obtenus'].sum()

Electoral district name
Abbotsford                                            48124
Abitibi--Baie-James--Nunavik--Eeyou                   34518
Abitibi--Témiscamingue                                49729
Acadie--Bathurst                                      50963
Ahuntsic-Cartierville                                 55567
Ajax                                                  56307
Alfred-Pellan                                         55169
Algoma--Manitoulin--Kapuskasing                       41374
Argenteuil--La Petite-Nation                          51066
Aurora--Oak Ridges--Richmond Hill                     50980
Avalon                                                42086
Avignon--La Mitis--Matane--Matapédia                  36356
Banff--Airdrie                                        66638
Barrie--Innisfil                                      49341
Barrie--Springwater--Oro-Medonte                      50531
Battle River--Crowfoot                                58769
Battlefords--Llo

In [16]:
idx = riding_df.groupby(DISTRICT_NAME)[VOTES].transform(max) == riding_df[VOTES]
fptp_winners = riding_df[idx]
fptp_winners.groupby(PARTY)['Province'].count()

Political affiliation
Bloc Québécois               10
Conservative                 99
Green Party                   1
Liberal                     184
NDP-New Democratic Party     44
Name: Province, dtype: int64

In [17]:
def dHondt(name, row, parties):
    seats = int(row['seats'])
    votes = [row[p] for p in parties]
    denominators = range(1,seats+1)
    coeffs = [[{ 'party' : parties[idx], 'coeff' : v/d} for d in denominators] for idx,v in enumerate(votes)]
    ranked_list = sorted([item for sublist in coeffs for item in sublist], key=lambda x:x['coeff'])
    winners = pd.DataFrame(ranked_list[-seats:])
    results = winners.groupby('party').count().rename(columns={'coeff':'seats'}).reset_index()
    results['Province'] = name
    return results

In [18]:
parties = riding_df[PARTY].unique()

In [19]:
riding_df[riding_df[DISTRICT_NAME] == 'Windsor West']

,% Votes obtained - Votes obtenus %,Appartenance politique,Electoral district name,Electoral district number - Numéro de la circonscription,Given name - Prénom,Middle name(s) - Autre(s) prénom(s),Nom de la circonscription,Political affiliation,Province,Rejected ballots - Bulletins rejetés***,Surname - Nom de famille,Total number of ballots cast - Nombre total de votes déposés,Type de résultats**,Type of results*,Votes obtained - Votes obtenus
33,20.8,Conservateur,Windsor West,35117,Henry,,Windsor-Ouest,Conservative,Ontario,284,Lau,47189,validés,validated,9734
34,2.3,Parti Vert,Windsor West,35117,Cora,,Windsor-Ouest,Green Party,Ontario,284,LaRussa,47189,validés,validated,1083
35,51.3,NPD-Nouveau Parti démocratique,Windsor West,35117,Brian,,Windsor-Ouest,NDP-New Democratic Party,Ontario,284,Masse,47189,validés,validated,24085
36,.3,Marxiste-Léniniste,Windsor West,35117,Margaret,,Windsor-Ouest,Marxist-Leninist,Ontario,284,Villamizar,47189,validés,validated,161
37,25.2,Libéral,Windsor West,35117,Dave,,Windsor-Ouest,Liberal,Ontario,284,Sundin,47189,validés,validated,11842


In [20]:
riding_pivot = riding_df.pivot_table(index=DISTRICT_NAME, columns=PARTY, values=VOTES).reset_index()
riding_pivot['seats'] = 1
riding_pivot.head()

Political affiliation,Electoral district name,ATN,Animal Alliance/Environment Voters,Bloc Québécois,CAP,Canada Party,Christian Heritage Party,Communist,Conservative,Democratic Advancement,...,No Affiliation,PACT,PC Party,Pirate,Radical Marijuana,Rhinoceros,Seniors Party,The Bridge,United Party,seats
0,Abbotsford,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23229,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,Abitibi--Baie-James--Nunavik--Eeyou,NaN,NaN,6398,NaN,NaN,NaN,NaN,3211,NaN,...,NaN,NaN,NaN,NaN,NaN,258,NaN,NaN,NaN,1
2,Abitibi--Témiscamingue,NaN,NaN,9651,NaN,NaN,NaN,NaN,3425,NaN,...,NaN,NaN,NaN,NaN,NaN,425,NaN,NaN,NaN,1
3,Acadie--Bathurst,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3852,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Ahuntsic-Cartierville,NaN,NaN,7346,NaN,NaN,NaN,NaN,4051,NaN,...,NaN,NaN,NaN,NaN,NaN,285,NaN,NaN,NaN,1


In [21]:
ridings_meta = pd.DataFrame([{DISTRICT_NAME : name, 'id' : r['id'], 'province': r['province']} for name, r in ridings.items()])
ridings_meta.head()

,Electoral district name,id,province
0,Repentigny,2129,Quebec
1,Prince George--Peace River--Northern Rockies,1697,British Columbia
2,Parkdale--High Park,2228,Ontario
3,New Brunswick Southwest,1589,New Brunswick
4,Brandon--Souris,1592,Manitoba


In [22]:
riding_totals = pd.merge(riding_pivot, ridings_meta, on=DISTRICT_NAME)
riding_totals.head()

Political affiliation,Electoral district name,ATN,Animal Alliance/Environment Voters,Bloc Québécois,CAP,Canada Party,Christian Heritage Party,Communist,Conservative,Democratic Advancement,...,PC Party,Pirate,Radical Marijuana,Rhinoceros,Seniors Party,The Bridge,United Party,seats,id,province
0,Abbotsford,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23229,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1674,Alberta
1,Abitibi--Baie-James--Nunavik--Eeyou,NaN,NaN,6398,NaN,NaN,NaN,NaN,3211,NaN,...,NaN,NaN,NaN,258,NaN,NaN,NaN,1,2070,Quebec
2,Abitibi--Témiscamingue,NaN,NaN,9651,NaN,NaN,NaN,NaN,3425,NaN,...,NaN,NaN,NaN,425,NaN,NaN,NaN,1,2071,Quebec
3,Acadie--Bathurst,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3852,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1582,New Brunswick
4,Ahuntsic-Cartierville,NaN,NaN,7346,NaN,NaN,NaN,NaN,4051,NaN,...,NaN,NaN,NaN,285,NaN,NaN,NaN,1,2072,Quebec


In [23]:
dHondt('national',riding_totals.sum(),parties)

,party,seats,Province
0,Bloc Québécois,16,national
1,Conservative,109,national
2,Green Party,11,national
3,Liberal,135,national
4,NDP-New Democratic Party,67,national


In [25]:
combine = riding_totals.groupby('province').sum().fillna(0)
provinces = [dHondt(region, r, parties) for region,r in combine.iterrows()]
pd.concat(provinces).groupby('party').sum().sort_values('seats', ascending=False)

,seats
party,
Liberal,143
Conservative,107
NDP-New Democratic Party,66
Bloc Québécois,15
Green Party,7
